In [1]:
from collections import OrderedDict
from collections import Counter
import numpy as np
import pandas
import heapq

In [3]:
visit_popularity = Counter()
purchase_popularity = Counter()

with open('train.txt', 'r') as f:
    for line in f:
        visits, purchases = line.strip().split(';')
        # TODO
        # update visit_popularity
        # update purchase_popularity
        for v in visits.split(','):
            visit_popularity[v] += 1
        for b in purchases.split(','):
            purchase_popularity[b] += 1

In [4]:
def recommend_by_purchase(items, max_count):
    return heapq.nlargest(max_count, OrderedDict.fromkeys(items), key=lambda x: purchase_popularity.get(x, 0))

def recommend_by_visit(items, max_count):
    return heapq.nlargest(max_count, OrderedDict.fromkeys(items), key=lambda x: visit_popularity.get(x, 0))

In [7]:
np.zeros(5)

array([ 0.,  0.,  0.,  0.,  0.])

In [31]:
def calculate_metrics(recommend, max_count, sessions_file):
    with open(sessions_file, 'r') as f:
        avg_recall = np.zeros(max_count)
        avg_precision = np.zeros(max_count)
        sessions_count = 0
        for line in f:
            visits, purchases = line.strip().split(';')
            if purchases != '':
                visits = visits.split(',')
                purchases = purchases.split(',')
                rec = recommend(visits, max_count)

                # TODO
                # update avg_recall
                # update avg_precision
                # update sessions_count
                
                rec_size = len(set(rec)) #k
                buy_size = len(set(purchases))
                
                buy_from_rec_size = len((set(rec) & set(purchases)))
                
                avg_recall[rec_size - 1] += buy_from_rec_size / buy_size  
                avg_precision[rec_size - 1] +=  buy_from_rec_size / rec_size

                sessions_count += 1

    return pandas.DataFrame({
            'k': np.arange(max_count) + 1,
            'avg_recall@k': [round(x, 2) for x in avg_recall / sessions_count],
            'avg_precision@k': [round(x, 2) for x in avg_precision / sessions_count]
    }).set_index('k')

In [32]:
calculate_metrics(recommend_by_purchase, 5, 'train.txt')

,avg_precision@k,avg_recall@k
k,,
1,0.20,0.19
2,0.09,0.17
3,0.05,0.11
4,0.03,0.09
5,0.12,0.36


In [33]:
calculate_metrics(recommend_by_purchase, 5, 'test.txt')

,avg_precision@k,avg_recall@k
k,,
1,0.20,0.19
2,0.10,0.18
3,0.05,0.11
4,0.03,0.08
5,0.08,0.26


In [34]:
calculate_metrics(recommend_by_visit, 5, 'train.txt')

,avg_precision@k,avg_recall@k
k,,
1,0.20,0.19
2,0.09,0.17
3,0.05,0.11
4,0.03,0.09
5,0.08,0.26


In [35]:
calculate_metrics(recommend_by_visit, 5, 'test.txt')

,avg_precision@k,avg_recall@k
k,,
1,0.20,0.19
2,0.10,0.18
3,0.05,0.11
4,0.03,0.08
5,0.08,0.24
